In [ ]:
from torchvision.datasets import OxfordIIITPet

oxford_pet = OxfordIIITPet("<dataset_loc>/OxfordPets", download=True, split="trainval")

imgs = []
labels = []
classes = oxford_pet.classes

for inst in oxford_pet:
    imgs.append(inst[0])
    labels.append(inst[1])

In [ ]:
dog_classes = [
 'American Bulldog',
 'American Pit Bull Terrier',
 'Basset Hound',
 'Beagle',
 'Boxer',
 'Chihuahua',
 'English Cocker Spaniel',
 'English Setter',
 'German Shorthaired',
 'Great Pyrenees',
 'Havanese',
 'Japanese Chin',
 'Keeshond',
 'Leonberger',
 'Miniature Pinscher',
 'Newfoundland',
 'Pomeranian',
 'Pug',
 'Saint Bernard',
 'Samoyed',
 'Scottish Terrier',
 'Shiba Inu',
 'Staffordshire Bull Terrier',
 'Wheaten Terrier',
 'Yorkshire Terrier']
cat_classes = list(set(classes) - set(dog_classes))


bin_classes = ['dog', 'cat']
binary_labels = [int(classes[x] not in dog_classes) for x in labels]

annotated_classes = [classes[x] + ' dog' if  classes[x] in dog_classes else classes[x] + ' cat' for x in labels]


In [ ]:
from alfred import Client

clip = Client(model_type="huggingfacevlm", model="openai/clip-vit-large-patch14")

In [ ]:
from alfred.template import ImageTemplate
from alfred.voter import Voter

import numpy as np

In [ ]:
label_prompt_0 = ImageTemplate({'label': classes}, "a photo of a [[label]]")
label_voter_0 = Voter(label_map=oxford_pet.class_to_idx)

label_prompt_1 = ImageTemplate({'ann_label': annotated_classes}, "a photo of a [[ann_label]]")
label_voter_1 = Voter(label_map={ann_label:idx for idx, ann_label in enumerate(annotated_classes)})

bin_prompt = ImageTemplate({'label': ['dog', 'cat']}, "a photo of a [[label]]")
bin_voter = Voter(label_map={'dog':0, 'cat':1})

voters = [label_voter_0, label_voter_1, bin_voter]

In [ ]:
votes = np.zeros([len(imgs), 3])
from voter in voters:
    prompts = templates[voter_idx].apply_to_dataset(imgs)
    resp = clip(prompts, batch_size=1024)
    votes[voter_idx,:] = voter.vote(resp)

In [ ]:
from alfred.labeling import NPLM


label_partition = {
    0: [[x+1] for x in range(len(classes))],
    1: [[x+1] for x in range(len(classes))],
    2: [[classes.index(x)+1 for x in dog_classes], [classes.index(x)+1 for x in cat_classes]]
}

nplm = NPLM(len(classes), label_partition, 'cpu')

In [ ]:
prob_label = nplm(votes)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(labels, prob_label.argmax(-1))